In [ ]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.101 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 41.2/235.7 GB disk)


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [ ]:
from ultralytics import YOLO
from torchinfo import summary

model = YOLO("yolov8n.pt")
summary(model.model.cuda(), input_size=(1, 3, 640, 640))

100%|██████████| 6.25M/6.25M [00:00<00:00, 381MB/s]


Layer (type:depth-idx)                             Output Shape              Param #
DetectionModel                                     [1, 84, 8400]             --
├─Sequential: 1-1                                  --                        --
│    └─Conv: 2-1                                   [1, 16, 320, 320]         --
│    │    └─Conv2d: 3-1                            [1, 16, 320, 320]         (432)
│    │    └─BatchNorm2d: 3-2                       [1, 16, 320, 320]         (32)
│    └─Detect: 2-96                                --                        (recursive)
│    │    └─ModuleList: 3-118                      --                        (recursive)
│    └─Conv: 2-3                                   [1, 32, 160, 160]         --
│    │    └─Conv2d: 3-4                            [1, 32, 160, 160]         (4,608)
│    │    └─BatchNorm2d: 3-5                       [1, 32, 160, 160]         (64)
│    └─Detect: 2-96                                --                        (recursi

In [ ]:
results_train = model.train(data='coco128.yaml', epochs=10, imgsz=640)

Ultralytics 8.3.101 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

100%|██████████| 6.66M/6.66M [00:00<00:00, 406MB/s]
Unzipping /content/datasets/coco128.zip to /content/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 4051.95file/s]

Dataset download success ✅ (1.7s), saved to /content/datasets



100%|██████████| 755k/755k [00:00<00:00, 103MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

...

                   all        128        929      0.671      0.658      0.704       0.53

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      3.41G      1.088      1.168      1.182        164        640: 100%|██████████| 8/8 [00:00<00:00, 10.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.98it/s]


                   all        128        929      0.664      0.661      0.706      0.531

10 epochs completed in 0.007 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.101 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.70it/s]


                   all        128        929      0.663      0.662      0.706      0.531
                person         61        254       0.78      0.701       0.78      0.571
               bicycle          3          6      0.856      0.333      0.367      0.324
                   car         12         46      0.691      0.239      0.325      0.201
            motorcycle          4          5      0.689      0.894      0.938      0.753
              airplane          5          6      0.712          1      0.955      0.783
                   bus          5          7      0.754      0.714      0.722       0.65
                 train          3          3      0.701          1      0.913      0.747
                 truck          5         12      0.857      0.499      0.528      0.374
                  boat          2          6      0.502      0.336      0.585      0.401
         traffic light          4         14      0.332      0.143      0.149      0.142
             stop sig

In [ ]:
results_val = model.val()

Ultralytics 8.3.101 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.62it/s]


                   all        128        929      0.679      0.656      0.708      0.534
                person         61        254      0.803      0.691      0.783      0.569
               bicycle          3          6      0.897      0.333      0.385      0.338
                   car         12         46      0.677      0.217      0.315      0.196
            motorcycle          4          5      0.688      0.889      0.938       0.77
              airplane          5          6      0.714          1      0.955      0.789
                   bus          5          7      0.703      0.714      0.722      0.648
                 train          3          3      0.705          1      0.913      0.747
                 truck          5         12      0.847      0.463      0.526      0.373
                  boat          2          6       0.55      0.333      0.518      0.376
         traffic light          4         14      0.447      0.143      0.149      0.142
             stop sig

In [ ]:
!ls datasets/coco128/images


train2017


In [ ]:
results_pred = model.predict(source="/content/datasets/coco128/images/val2017", conf=0.25)

for result in results_pred:
    result.show()


print("Result Train：", results_train)
print("Result Validation：", results_val)

FileNotFoundError: /content/datasets/coco128/images/val2017 does not exist